In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/comments/merged_df-2.csv


In [2]:
!pip install langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=e5d24426ca626168770b0e34e55dbfde85f0c074cb084099c9281927dfd9b4ff
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [3]:
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
from langdetect import detect, DetectorFactory

DetectorFactory.seed = 0

nltk.download('vader_lexicon')

# Load data
df = pd.read_csv("/kaggle/input/comments/merged_df-2.csv")
df['extracted_comments'] = df['extracted_comments'].fillna("")

# Initialize VADER
sia = SentimentIntensityAnalyzer()

# Function: Detect language and apply sentiment
def get_vader_sentiment(text):
    try:
        if detect(text) == 'en':
            return sia.polarity_scores(text)['compound']
        else:
            return 0  # Non-English
    except:
        return 0  # Empty or undetectable text

# Apply sentiment analysis
df['sentiment_score'] = df['extracted_comments'].apply(get_vader_sentiment)

# Average sentiment per influencer
influencer_scores = df.groupby('User').agg({
    'sentiment_score': 'mean',
    'extracted_comments': lambda x: list(x)[:3]  # first 3 comments for display
}).reset_index()

# Sort by sentiment (ascending)
influencer_scores = influencer_scores.sort_values(by='sentiment_score', ascending=True)

for index, row in influencer_scores.head(25).iterrows():
    print(f"Influencer: {row['User']}")
    print(f"Average Sentiment Score: {row['sentiment_score']:.3f}")
    print("Sample Comments:")
    for comment in row['extracted_comments']:
        print(f" - {comment}")
    print("-" * 50)

influencer_scores.to_csv("influencer_sentiment_scores.csv", index=False)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Influencer: kelseyplum10
Average Sentiment Score: -0.998
Sample Comments:
 - Anyone else 👀 those kyries tho???? 🔥🔥🔥I have the same pair 😆 Is that an airbrush tattoo ? Work it out young lady I'm gonna be at UW Thursday, but nobody is giving us a tour of the athletics... smh cuz I play for Sol right now and I'm trying to get a look at what the facilities are like there.  Kelsey wake the fuck up and read my comments and request to follow  right now and see her posts and stop ignoring me Mal !!  kelsey wake the fuck up and request to follow  right now and see her posts and stop ignoring me and read my comments and pay attention to my comments please girl   best water out there!  kelsey why the fuck aren't you even reading my comments at all, are you blind or what, please read my comments and please request to follow  and see her posts and stop ignoring me Work  Kelsey come on stop being blind and read my fucking comments and request to follow  and see her posts  Kelsey wake up and please f

In [4]:
influencer_scores.head()

,User,sentiment_score,extracted_comments
10396,kelseyplum10,-0.9983,[Anyone else 👀 those kyries tho???? 🔥🔥🔥I have ...
12086,lovenhiphopfans,-0.9979,"[Y’all could’ve used a better picture, Home gi..."
16466,randecent,-0.9968,[At that point not gonna front I would make a ...
20151,thecomputernerd01,-0.9961,[So no head? When you studied for a test but y...
22634,whatsinside,-0.9906,"[ too much money, and nobody else would do tha..."


In [5]:
# Save only 'User' and 'sentiment_score' columns to CSV
influencer_scores[['User', 'sentiment_score']].to_csv("influencer_sentiment_scores_final.csv", index=False)
